# 🧠 Alzheimer CNN Sınıflandırması: Basit CNN Modeli (Baseline)

Bu notebook, transfer öğrenme gibi gelişmiş teknikler kullanmadan, temel bir **Convolutional Neural Network (CNN)** modeliyle projemize bir başlangıç noktası (`baseline`) oluşturmayı amaçlamaktadır.

### 1. Model Mimarisi

Kullanılan basit CNN modeli, görüntüden özellik çıkarmak için evrişimsel katmanları, boyutu küçültmek için havuzlama katmanlarını ve tahmin yapmak için tam bağlı katmanları içerir.

In [ ]:
class AlzheimerCNN(nn.Module):
    def __init__(self, num_classes: int = 4):
        super(AlzheimerCNN, self).__init__()

        # 1. Convolutional block
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 2. Convolutional block
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 3. Convolutional block
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1   = nn.Linear(128 * 28 * 28, 512)  # Input boyutu image size'a göre değişebilir!
        self.drop1 = nn.Dropout(p=0.5)
        self.fc2   = nn.Linear(512, num_classes)

    def forward(self, x):
        # Block 1
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        # Block 2
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        # Block 3
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        # Flatten
        x = x.view(x.size(0), -1)
        # Fully connected
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = self.fc2(x)
        return x


### 2. Eğitim ve Doğrulama Eğrileri

Modelin eğitim ve doğrulama süreçleri incelendiğinde, temel bir CNN'nin öğrenme dinamikleri net bir şekilde görülmektedir.

![Training and Validation Loss and Accuracy](images/training_curvesv2.png)

#### **Analiz:**
* **Eğitim ve Doğrulama Kaybı (`Loss`):** Eğitim kaybı istikrarlı bir şekilde azalırken, doğrulama kaybı yüksek seviyelerde dalgalanmalar gösteriyor. Bu durum, modelin eğitim verisini ezberleme eğiliminde olduğunu ve yeni verilere genelleme yapmakta zorlandığını gösterir.
* **Eğitim ve Doğrulama Doğruluğu (`Accuracy`):** Doğrulama doğruluğu, en yüksek **%58** seviyesine ulaşsa da, istikrarsızdır ve dalgalanmalar gösterir. Eğitim doğruluğu ise daha yavaş yükselir, bu da modelin öğrenme kapasitesinin sınırlı olduğunu işaret eder. Bu grafikler, modelin aşırı öğrenme (`overfitting`) problemi yaşadığını açıkça göstermektedir.

### 3. Sınıflandırma Metrikleri

Temel modelin doğrulama seti üzerindeki performansını daha yakından incelemek, sınıflar arası denge sorununu ortaya koyar.

![Confusion Matrix](images/confusion_matrixv2.png)

#### **Analiz:**
* **Genel Doğruluk:** %52 gibi düşük bir doğruluk değeri, temel modelin Alzheimer evrelerini güvenilir bir şekilde sınıflandıramadığını gösterir.
* **`ModerateDemented` Sınıfı:** Karışıklık matrisinde görüldüğü gibi, en nadir sınıf olan `ModerateDemented` için **hiçbir doğru tahmin yapılamamıştır**. Model bu sınıfı tamamen göz ardı etmiştir.
* **`VeryMildDemented` Sınıfı:** 359 örneğin sadece 92'si doğru tahmin edilmiştir. Geri kalan 267 örnek `MildDemented` olarak yanlış sınıflandırılmıştır.
* **Sınıf Dengesi Sorunu:** Model, `MildDemented` sınıfına 144 adet, `VeryMildDemented` sınıfına 92 adet, `NonDemented` sınıfına 339 adet yanlış tahmin yapmıştır. Bu, modelin veri setindeki dengesizliği yönetemediğini ve tahminlerini çoğunluk sınıflara kaydırdığını göstermektedir.

### 4. Grad-CAM

![Grad-CAM Heatmap](images/gradcamv2.png)

#### **Analiz:**
* Grad-CAM görüntüsü, modelin beynin genel yapısına değil, daha çok kenar bölgelerine odaklandığını göstermektedir. Bu, modelin sınıflar arasındaki farkı, anatomik bilgilerden ziyade, görüntünün yüzeysel özelliklerinden öğrenmeye çalıştığını işaret eder.